In [5]:
import pandas as pd
import statsmodels.api as sm
import itertools
import joblib
import warnings
import os

def train_sarima_model_manual():

    master_dataset_path = 'reliance_master_dataset.csv'
    output_model_path = 'reliance_sarima_model_tuned.pkl'
    print("Starting Manual SARIMA Model Tuning and Training for Reliance Industries")

    try:
        print(f"Loading data from {master_dataset_path}...")
        if not os.path.exists(master_dataset_path):
            raise FileNotFoundError(f"Master dataset not found: {master_dataset_path}")

        df = pd.read_csv(master_dataset_path)
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)

        price_data = df['close']
        print("Data loaded successfully.")

        p = d = q = range(0, 2)
        pdq = list(itertools.product(p, d, q)) 

        # Seasonal component. m=5 for 5-day trading week (Monday-Friday)
        seasonal_pdq = [(x[0], x[1], x[2], 5) for x in list(itertools.product(p, d, q))]

        print("\nSearching for the best SARIMA parameters via grid search...")
        print("Parameter combinations to test:", len(pdq) * len(seasonal_pdq))

        best_aic = float("inf") # Initialize best AIC with infinity
        best_pdq = None
        best_seasonal_pdq = None


        for param in pdq:
            for param_seasonal in seasonal_pdq:
                try:
                    mod = sm.tsa.statespace.SARIMAX(price_data,
                                                    order=param,
                                                    seasonal_order=param_seasonal,
                                                    enforce_stationarity=False, 
                                                    enforce_invertibility=False)
                    results = mod.fit(disp=False)

                    if results.aic < best_aic:
                        best_aic = results.aic
                        best_pdq = param
                        best_seasonal_pdq = param_seasonal
                        print(f"New best model found -> AIC: {results.aic:.2f} | Order: {param} | Seasonal Order: {param_seasonal}")

                except Exception as e:
                    continue

        if best_pdq is None:
            print("No suitable SARIMA model found within the defined parameter space. Consider expanding the search space or checking data.")
            return

        print("\n--- Grid Search Complete ---")
        print(f"Best SARIMA Model Found:")
        print(f"AIC: {best_aic:.2f}")
        print(f"Order (p,d,q): {best_pdq}")
        print(f"Seasonal Order (P,D,Q,m): {best_seasonal_pdq}")

        print("\nFitting the final model with the best parameters...")
        final_model = sm.tsa.statespace.SARIMAX(price_data,
                                                order=best_pdq,
                                                seasonal_order=best_seasonal_pdq,
                                                enforce_stationarity=False,
                                                enforce_invertibility=False)
        final_results = final_model.fit(disp=False)

        print(f"\nSaving the tuned model to {output_model_path}...")

        joblib.dump(final_results, output_model_path)
        print("Model saved successfully.")

    except FileNotFoundError as e:
        print(f"Error: {e}")
        print("Please ensure the previous step (master dataset creation) was completed successfully.")
    except Exception as e:
        print(f"An unexpected error occurred during SARIMA model training: {e}")

    print("\n--- SARIMA Model Training Completed ---")

if __name__ == '__main__':
    train_sarima_model_manual()


--- Starting Manual SARIMA Model Tuning and Training for Reliance Industries ---
Loading data from reliance_master_dataset.csv...
Data loaded successfully.

Searching for the best SARIMA parameters via grid search...
Parameter combinations to test: 64
New best model found -> AIC: 76868.75 | Order: (0, 0, 0) | Seasonal Order: (0, 0, 0, 5)
New best model found -> AIC: 70295.34 | Order: (0, 0, 0) | Seasonal Order: (0, 0, 1, 5)
New best model found -> AIC: 44311.49 | Order: (0, 0, 0) | Seasonal Order: (0, 1, 0, 5)
New best model found -> AIC: 44265.63 | Order: (0, 0, 0) | Seasonal Order: (0, 1, 1, 5)
New best model found -> AIC: 41693.23 | Order: (0, 0, 1) | Seasonal Order: (0, 1, 0, 5)
New best model found -> AIC: 41447.78 | Order: (0, 0, 1) | Seasonal Order: (0, 1, 1, 5)
New best model found -> AIC: 41441.77 | Order: (0, 0, 1) | Seasonal Order: (1, 1, 1, 5)
New best model found -> AIC: 36988.38 | Order: (0, 1, 0) | Seasonal Order: (0, 0, 0, 5)
New best model found -> AIC: 36957.51 | Orde